In [50]:
#Importing relevant libraries
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds 

In [51]:
#uploading mnist dataset already exisitng in tensorflow library
mnist_data, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised = True)

mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

#creating the number of validation dataset from the training dataset
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples

#convert number of the validation sample to an integer to prevent potential issue
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

#get the number data point in our  test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


#scaling the training data 
def scale(image,label):
    image = tf.cast(image, tf.float32)
    image/= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

#using the scale function to scale the test data
test_data = mnist_test.map(scale)

#shuffle the input in batches

BUFFER_SIZE = 10000

shuffle_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
#create the validation data
validation_data = shuffle_train_and_validation_data.take(num_validation_samples)

#create the train data
train_data = shuffle_train_and_validation_data.skip(num_validation_samples)

#batching the training data

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

#No need to batch the validation and test data(we are not backpropagating on them ). Hence we will take all at once]

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


In [52]:
#Buildimg the model

input_size = 784
output_size = 10
hidden_layer_size = 200


model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
       
        tf.keras.layers.Dense(output_size, activation='softmax')
        ])

In [53]:
# Choose the Optimizer and the loss function

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
#number of epoch
NUM_EPOCH = 5

model.fit(train_data, epochs = NUM_EPOCH,validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 7s - loss: 0.2612 - accuracy: 0.9231 - val_loss: 0.1377 - val_accuracy: 0.9598 - 7s/epoch - 14ms/step
Epoch 2/5
540/540 - 9s - loss: 0.1010 - accuracy: 0.9695 - val_loss: 0.0867 - val_accuracy: 0.9748 - 9s/epoch - 17ms/step
Epoch 3/5
540/540 - 10s - loss: 0.0695 - accuracy: 0.9789 - val_loss: 0.0696 - val_accuracy: 0.9773 - 10s/epoch - 18ms/step
Epoch 4/5
540/540 - 9s - loss: 0.0543 - accuracy: 0.9824 - val_loss: 0.0650 - val_accuracy: 0.9792 - 9s/epoch - 16ms/step
Epoch 5/5
540/540 - 10s - loss: 0.0382 - accuracy: 0.9881 - val_loss: 0.0437 - val_accuracy: 0.9852 - 10s/epoch - 18ms/step


In [55]:
#Test the Model

test_loss, test_accuracy = model.evaluate(test_data)

print('Test loss: {0:.2f}. Test accuracy:{1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 1s 730ms/step - loss: 0.0739 - accuracy: 0.9788
Test loss: 0.07. Test accuracy:97.88%
